SECCIÓN 1 — Instalación de dependencias

In [1]:
!pip install librosa gdown soundfile tensorflow_hub


SECCIÓN 2 — Importaciones y Configuración del Dataset

In [2]:
import numpy as np
import librosa
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow_hub as hub
import soundfile as sf
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
import gdown
import zipfile


SECCIÓN 3 — Descarga y extracción del Dataset
📌 Se organiza automáticamente en carpetas por género

In [3]:
# ID del archivo GTZAN en tu Google Drive
file_id = "1XP20Z-T-Q_7uvyXjPS7-aIxW_8XRhYo9"
url = f"https://drive.google.com/uc?id={file_id}"

output_zip = "/content/genres.zip"

print("📥 Descargando dataset...")
gdown.download(url, output_zip, quiet=False)

print("📂 Extrayendo...")
with zipfile.ZipFile(output_zip, 'r') as zip_ref:
    zip_ref.extractall('/content/')

DATA_PATH = "/content/genres"

GENRES = ['blues','classical','country','disco','hiphop',
          'jazz','metal','pop','reggae','rock']


📥 Descargando dataset...


Downloading...
From (original): https://drive.google.com/uc?id=1XP20Z-T-Q_7uvyXjPS7-aIxW_8XRhYo9
From (redirected): https://drive.google.com/uc?id=1XP20Z-T-Q_7uvyXjPS7-aIxW_8XRhYo9&confirm=t&uuid=52e852ed-e54f-4247-813c-bcb8a163b231
To: /content/genres.zip
100%|██████████| 1.23G/1.23G [00:10<00:00, 118MB/s]


📂 Extrayendo...


SECCIÓN 4 — Modelo YAMNet para extraer embeddings

📌 Convierte audio → vector 1024 dimensiones

In [4]:
yamnet = hub.load('https://tfhub.dev/google/yamnet/1')

def extract_yamnet_embeddings(file_path):
    try:
        audio, sr = sf.read(file_path, dtype='float32')

        if sr != 16000:
            audio = librosa.resample(audio, orig_sr=sr, target_sr=16000)

        _, embeddings, _ = yamnet(audio)
        return np.mean(embeddings.numpy(), axis=0)

    except Exception as e:
        print("Error:", e)
        return None


SECCIÓN 5 — Carga del dataset y extracción de embeddings

In [5]:
print("🎧 Extrayendo embeddings...")

features = []
labels = []

for genre in GENRES:
    folder = os.path.join(DATA_PATH, genre)
    files = [f for f in os.listdir(folder) if f.endswith(".au")]

    for f in files:
        fp = os.path.join(folder, f)
        emb = extract_yamnet_embeddings(fp)

        if emb is not None:
            features.append(emb)
            labels.append(genre)

X = np.array(features)
y = np.array(labels)

print("✔ Listo!")
print("X shape:", X.shape)
print("Samples:", len(y))

🎧 Extrayendo embeddings...
✔ Listo!
X shape: (1000, 1024)
Samples: 1000


SECCIÓN 6 — Codificación y División Train/Test

In [6]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_cat = to_categorical(y_encoded)

X_train, X_test, y_train, y_test = train_test_split(
    X, y_cat, test_size=0.2, random_state=42, stratify=y_encoded
)

print("📊 Train:", X_train.shape, "| Test:", X_test.shape)

📊 Train: (800, 1024) | Test: (200, 1024)


SECCIÓN 7 — Modelo de Clasificación

📌 Entrada: 1024 features → Salida: 10 géneros

In [7]:
model = Sequential([
    Dense(512, activation='relu', input_shape=(1024,)),
    Dropout(0.3),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(len(GENRES), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 658,698 (2.51 MB)

 Trainable params: 658,698 (2.51 MB)

 Non-trainable params: 0 (0.00 B)

SECCIÓN 8 — Entrenamiento del modelo

In [8]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=40,
    batch_size=32,
    verbose=1
)


Epoch 1/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - accuracy: 0.4662 - loss: 1.7718 - val_accuracy: 0.8350 - val_loss: 0.5946
Epoch 2/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.7772 - loss: 0.6704 - val_accuracy: 0.8550 - val_loss: 0.4790
Epoch 3/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.7890 - loss: 0.5634 - val_accuracy: 0.8700 - val_loss: 0.4573
Epoch 4/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.8228 - loss: 0.5133 - val_accuracy: 0.8800 - val_loss: 0.4281
Epoch 5/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.8558 - loss: 0.4390 - val_accuracy: 0.8600 - val_loss: 0.4354
Epoch 6/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8441 - loss: 0.4663 - val_accuracy: 0.8950 - val_loss: 0.4098
Epoch 7/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8635 - loss: 0.4091 - val_accuracy: 0.9050 - val_loss: 0.3988
Epoch 8/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.8860 - loss: 0.3655 - val_accuracy: 0.8950 - v

SECCIÓN 9 — Evaluación

In [9]:
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"\n🎯 Accuracy en test: {acc*100:.2f}%")



🎯 Accuracy en test: 88.00%


SECCIÓN 10 — Predicción desde audio real + Reproductor

In [10]:
import ipywidgets as widgets
from IPython.display import Audio, display, clear_output
import random

def test_random_audio(_=None):
    clear_output(wait=True)
    print("🎲 Seleccionando audio aleatorio...")

    genre = random.choice(GENRES)
    folder = os.path.join(DATA_PATH, genre)
    files = [f for f in os.listdir(folder) if f.endswith(".au")]

    file = random.choice(files)
    fp = os.path.join(folder, file)

    print(f"\n🗂 Archivo: {file}")
    print(f"🎵 Género real: {genre}")

    # Reproducir audio
    audio, sr = librosa.load(fp, sr=None)
    display(Audio(audio, rate=sr))

    emb = extract_yamnet_embeddings(fp)
    emb = emb.reshape(1, -1)

    pred = model.predict(emb, verbose=0)[0]
    idx = np.argmax(pred)
    pred_genre = label_encoder.inverse_transform([idx])[0]

    print(f"\n🔮 Predicción: {pred_genre}")
    print("-----------------------------------")

    button = widgets.Button(description="▶ Otro audio", button_style="success")
    button.on_click(test_random_audio)
    display(button)

test_random_audio()


🎲 Seleccionando audio aleatorio...

🗂 Archivo: hiphop.00058.au
🎵 Género real: hiphop



🔮 Predicción: hiphop
-----------------------------------


Button(button_style='success', description='▶ Otro audio', style=ButtonStyle())